In [ ]:
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-

%cd C:\Users\marci\Desktop\bookomend

from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers
import sqlite3
import pandas as pd
import bcrypt



log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


secretKey = "EOXy5MWDtObqlWnLWAmoMuQVaLXiMh2X"


app = Bottle() 

@app.route('/static/:path#.+#', name='static')
def static(path):
    return static_file(path, root='./static')

class User():
    def __init__(self, email):
        self.email = email
        
    def get_password(self):
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("SELECT Password FROM Users WHERE email = ?", (self.email,))
        password = cur.fetchall()
        con.close()
        if password:
            password = password[0][0]
        return password
    
    def get_randStr(self):
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("SELECT randStr FROM Users WHERE email = ?", (self.email,))
        randStr = cur.fetchall()
        con.close()
        if randStr:
            randStr = randStr[0][0]
        return randStr
    
    def get_loggedIn(self):
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("SELECT Logged_In FROM Users WHERE email = ?", (self.email,))
        logged_in = cur.fetchall()
        con.close()
        if logged_in:
            logged_in = logged_in[0][0]
        return logged_in
    
    def get_lastSeen(self):
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("SELECT Last_Seen FROM Users WHERE email = ?", (self.email,))
        last_seen = cur.fetchall()
        con.close()
        if last_seen:
            last_seen = last_seen[0][0]
        return last_seen
    
    def get_username(self):
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("SELECT Username FROM Users WHERE email = ?", (self.email,))
        username = cur.fetchall()
        con.close()
        if username:
            username = username[0][0]
        return username


    
def checkAuth():
    loginName = request.get_cookie("user", secret=secretKey)
    randStr = request.get_cookie("randStr", secret=secretKey)
    
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    user = User(loginName)
    
    if user.get_loggedIn() and user.get_randStr() == randStr and user.get_loggedIn() == "True":
        name = user.get_username()
        return name
    return redirect('/login')



@app.route('/login')
@app.route('/login/')
@app.route('/login', method='POST')
def login():
    loginName = request.forms.get('login_name', default=False)
    password = request.forms.get('password', default=False)
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    
    user = User(loginName)
    
    if(loginName != False and password != False):
        hashed = user.get_password()
        name = user.get_username()
        password_bytes = password.encode("utf-8")
    else:
        hashed = None
        name = None

    
    if(user.get_username() and bcrypt.checkpw(password_bytes, hashed)):
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)
        
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("UPDATE Users SET Logged_In = ?, randStr = ?, Last_Seen = ? \
                    WHERE email = ?", ("True", randStr, time.time(), loginName))
        con.commit()
        con.close()
        
        redirect('/recommendationform')
        return True
    else:
        return template('login')
    return template('login')


@app.route('/')
@app.route('/index')
@app.route('/index/')
def index():
    name = checkAuth()
    return template('index.tpl', name = name)


@app.route('/registration/')
@app.route('/registration', method=['POST', 'GET'])
def registration():
    message = ""
      
    email = request.forms.get('email', default=False)
    username = request.forms.get('username', default=False)
        
    password = request.forms.get('password', default="False").encode("utf-8")
    hashed = bcrypt.hashpw(password, bcrypt.gensalt())
        
    name = request.forms.get('name', default=False)
    surname = request.forms.get('surname', default=False)
        
    con = sqlite3.connect("book-o-mend database.db")
    cur = con.cursor()
        
    cur.execute("SELECT email, Username FROM Users WHERE email=? OR Username=?", (email,username))
    results = cur.fetchall()

    if(len([item for item in results if email in item]) > 0):
        message = "There is an account registered for this e-mail adress already. Forgot your password?"
        return template('registration_form.tpl', message = message)
    elif(len([item for item in results if username in item]) > 0):
        message = "This username is already taken. Please choose another one"
        return template('registration_form.tpl', message = message)
    else:
        message = ""

    if(email is not False):    
        cur.execute("""INSERT INTO Users (Username, Password, Name, Surname, email) 
                    VALUES (?, ?, ?, ?, ?)""", (username, hashed, name, surname, email))
        con.commit()
        con.close()
        return redirect('/login')
    
    else:
        return template('registration_form.tpl', message = message)
    
    
@app.route('/logout')
@app.route('/logout/')
def logout():
    loginName = request.get_cookie("user", secret=secretKey)
    response.delete_cookie("user", secret=secretKey)
    response.delete_cookie("randStr", secret=secretKey)
    
    con = sqlite3.connect("book-o-mend database.db")
    cur = con.cursor()
    cur.execute("UPDATE Users SET Logged_In = 'False' WHERE email = ?", (loginName, ))
    con.commit()
    con.close()
        
    redirect('/')
    
    
    
@app.route('/stats')
@app.route('/stats/')
def stats():
    checkAuth()
    print("KURWA DZIALAJ")
    email = request.get_cookie("user", secret=secretKey)
    con = sqlite3.connect("book-o-mend database.db")
    cur = con.cursor()
    
    query = "SELECT SUM(Number_of_Pages), COUNT(DISTINCT Books_Read.ID_Book) \
            FROM Books_Read \
            LEFT JOIN Books on Books_Read.ID_Book = Books.ID_Book \
            LEFT JOIN Users on Books_Read.ID_User = Users.ID_User WHERE email = ?"
    cur.execute(query, (email,))
    result = cur.fetchall()
    sum_of_pages = result[0][0]
    number_of_books = result[0][1]

    
    
    
    cur.execute("""SELECT Genre_Name, COUNT(DISTINCT Books_Read.ID_Book) as Genre_Count 
                FROM Books_Read LEFT JOIN Books on Books_Read.ID_Book = Books.ID_Book
                LEFT JOIN Genres on Genres.ID_Genre = Books.ID_Genre
                LEFT JOIN Users on Books_Read.ID_User = Users.ID_User
                WHERE email = ?
                GROUP BY Genre_Name
                ORDER BY Genre_Count
                DESC LIMIT 1""", (email,))
    
    fav_genre = cur.fetchall()
    if fav_genre:
        fav_genre_name = fav_genre[0][0]
        fav_genre_number = fav_genre[0][1]
    
    cur.execute("""SELECT Genre_Name, COUNT(DISTINCT Books_Read.ID_Book) as Genre_Count
                FROM Books_Read \
                LEFT JOIN Books on Books_Read.ID_Book = Books.ID_Book
                LEFT JOIN Genres on Genres.ID_Genre = Books.ID_Genre
                LEFT JOIN Users on Books_Read.ID_User = Users.ID_User
                WHERE email = ?
                GROUP BY Genre_Name
                ORDER BY Genre_Count
                DESC LIMIT 5""", (email,))
    fav_genres = cur.fetchall()
    
    
    
    cur.execute("SELECT Book_Author, COUNT(DISTINCT Books_Read.ID_Book) as Author_Count \
                FROM Books_Read \
                LEFT JOIN Books on Books_Read.ID_Book = Books.ID_Book \
                LEFT JOIN Users on Books_Read.ID_User = Users.ID_User \
                WHERE email = ? \
                GROUP BY Book_Author \
                ORDER BY Author_Count \
                DESC LIMIT 1", (email,))
    fav_author = cur.fetchall()
    
    if fav_author:
        fav_author_name = fav_author[0][0]
        fav_author_number = fav_author[0][1]
    
    cur.execute("SELECT Book_Author, COUNT(DISTINCT Books_Read.ID_Book) as Author_Count \
                FROM Books_Read \
                LEFT JOIN Books on Books_Read.ID_Book = Books.ID_Book \
                LEFT JOIN Users on Books_Read.ID_User = Users.ID_User \
                WHERE email = ? \
                GROUP BY Book_Author \
                ORDER BY Author_Count \
                DESC LIMIT 1", (email,))
    fav_authors = cur.fetchall()
    
    
    
    cur.execute("SELECT Book_Title, MAX(Average_Rating) \
                FROM Books_Read \
                LEFT JOIN Books on Books_Read.ID_Book = Books.ID_Book \
                LEFT JOIN Users on Books_Read.ID_User = Users.ID_User \
                LEFT JOIN Ratings on Books_Read.ID_Book = Ratings.ID_Book \
                WHERE email = ?", (email,))
    rating = cur.fetchall()
    
    con.close()
    if rating:
        highest_rating = rating[0][1]
        best_rated_book = rating[0][0]
    
    if not number_of_books:
        return "<h3> You haven't read any books yet </h3>"
    
    print(sum_of_pages, number_of_books, fav_genre_name , fav_genre_number, 
                    fav_author_name, fav_author_number)
    return template('stats.tpl', sum_of_pages = sum_of_pages, number_of_books = number_of_books, fav_genre_name = fav_genre_name, fav_genre_number = fav_genre_number, 
                    fav_author_name = fav_author_name, fav_author_number = fav_author_number)
    

    
    
@app.route('/recommendationform')
@app.route('/recommendationform/')
@app.route('/recommendationform', method=['POST', 'GET'])
def recommendationform():
    checkAuth()
    Title_of_the_book = request.forms.get('Book_Title', default = False)
    Type_of_genre = request.forms.get('Genre_Name', default = False)
    Number_of_pages = request.forms.get('Number_of_Pages', default = False)
    Standalone_or_a_series = request.forms.get('Is_Series', default = False)
    
    def check_recommendationform(Title_of_the_book, Type_of_genre, Number_of_pages, Standalone_or_a_series):
        if Title_of_the_book is not False and Type_of_genre is not False and Number_of_pages is not False and Standalone_or_a_series is not False:
            return True
        else:
            return False
    
    if check_recommendationform(Title_of_the_book, Type_of_genre, Number_of_pages, Standalone_or_a_series) is True:    
        con = sqlite3.connect("book-o-mend database.db")
        cur = con.cursor()
        cur.execute("SELECT * FROM Recommendations \
                    WHERE ID_Book in (SELECT ID_Book FROM Books WHERE Book_Title LIKE ?)", ('%'+Title_of_the_book+'%', ))
        recommendations = cur.fetchall()
        

        if Type_of_genre == "yes":
            recommendations = recommendations[0] + (Number_of_pages,) + ('%'+Title_of_the_book+'%',) + ('%'+Title_of_the_book+'%',)
            if Standalone_or_a_series == "TRUE":
                cur.execute("""SELECT Book_Title, Book_Author, Number_of_Pages, Publication_Year, Goodreads_Listing, Book_Description, Genre_Name, Average_rating, Book_Cover_Link
                FROM Books
                LEFT JOIN Genres ON Books.ID_Genre = Genres.ID_Genre
                LEFT JOIN If_series ON Books.ID_BOOK = If_series.ID_Book
                LEFT JOIN Ratings ON Books.ID_Book = Ratings.ID_Book
                WHERE Books.ID_Book in (?, ?, ?, ?, ?, ?)
                AND Number_of_Pages < ?
                AND Genre_Name in (SELECT Genre_Name
                FROM Books
                LEFT JOIN Genres on Books.ID_Genre = Genres.ID_Genre
                WHERE Book_Title LIKE ?)
                AND Books.Is_Series = 'TRUE' AND Book_Title NOT LIKE ?
                ORDER BY RANDOM()
                LIMIT 1""", recommendations)
                results = cur.fetchall()
            elif Standalone_or_a_series == "FALSE": 
                cur.execute("SELECT Book_Title, Book_Author, Number_of_Pages, Publication_Year, Goodreads_Listing, Book_Description, Genre_Name, Average_rating, Book_Cover_Link \
                FROM Books \
                LEFT JOIN Genres ON Books.ID_Genre = Genres.ID_Genre \
                LEFT JOIN If_series ON Books.ID_BOOK = If_series.ID_Book \
                LEFT JOIN Ratings ON Books.ID_Book = Ratings.ID_Book \
                WHERE Books.ID_Book in (?, ?, ?, ?, ?, ?) \
                AND Number_of_Pages < ? \
                AND Genre_Name in (SELECT Genre_Name \
                FROM Books \
                LEFT JOIN Genres on Books.ID_Genre = Genres.ID_Genre \
                WHERE Book_Title LIKE ?) \
                AND Books.Is_Series = 'FALSE'\
                AND Book_Title NOT LIKE ? \
                ORDER BY RANDOM() \
                LIMIT 1", recommendations)
                results = cur.fetchall()
            else: 
                cur.execute("SELECT Book_Title, Book_Author, Number_of_Pages, Publication_Year, Goodreads_Listing, Book_Description, Genre_Name, Average_rating, Book_Cover_Link \
                FROM Books \
                LEFT JOIN Genres ON Books.ID_Genre = Genres.ID_Genre \
                LEFT JOIN Ratings ON Books.ID_Book = Ratings.ID_Book \
                WHERE Books.ID_Book in (?, ?, ?, ?, ?, ?) \
                AND Number_of_Pages < ? \
                AND Genre_Name in (SELECT Genre_Name \
                FROM Books \
                LEFT JOIN Genres on Books.ID_Genre = Genres.ID_Genre \
                WHERE Book_Title LIKE ?) \
                AND Book_Title NOT LIKE ? \
                ORDER BY RANDOM() \
                LIMIT 1", recommendations)
                results = cur.fetchall()
            
        else:
            recommendations = recommendations[0] + (Number_of_pages,) + ('%'+Title_of_the_book+'%',)
            if Standalone_or_a_series == "TRUE":
                cur.execute("SELECT Book_Title, Book_Author, Number_of_Pages, Publication_Year, Goodreads_Listing, Book_Description, Genre_Name, Average_rating, Book_Cover_Link \
                FROM Books \
                LEFT JOIN Genres ON Books.ID_Genre = Genres.ID_Genre \
                LEFT JOIN If_series ON Books.ID_BOOK = If_series.ID_Book \
                LEFT JOIN Ratings ON Books.ID_Book = Ratings.ID_Book \
                WHERE Books.ID_Book in (?, ?, ?, ?, ?, ?) \
                AND Number_of_Pages < ? \
                AND Books.Is_Series = 'TRUE'\
                AND Book_Title NOT LIKE ? \
                ORDER BY RANDOM() \
                LIMIT 1", recommendations)
                results = cur.fetchall()
            elif Standalone_or_a_series == "FALSE": 
                cur.execute("SELECT Book_Title, Book_Author, Number_of_Pages, Publication_Year, Goodreads_Listing, Book_Description, Genre_Name, Average_rating, Book_Cover_Link \
                FROM Books \
                LEFT JOIN Genres ON Books.ID_Genre = Genres.ID_Genre \
                LEFT JOIN If_series ON Books.ID_BOOK = If_series.ID_Book \
                LEFT JOIN Ratings ON Books.ID_Book = Ratings.ID_Book \
                WHERE Books.ID_Book in (?, ?, ?, ?, ?, ?) \
                AND Number_of_Pages < ? \
                AND Books.Is_Series = 'FALSE'\
                AND Book_Title NOT LIKE ? \
                ORDER BY RANDOM() \
                LIMIT 1", recommendations)
                results = cur.fetchall()
            else: 
                cur.execute("SELECT Book_Title, Book_Author, Number_of_Pages, Publication_Year, Goodreads_Listing, Book_Description, Genre_Name, Average_rating, Book_Cover_Link \
                FROM Books \
                LEFT JOIN Genres ON Books.ID_Genre = Genres.ID_Genre \
                LEFT JOIN Ratings ON Books.ID_Book = Ratings.ID_Book \
                WHERE Books.ID_Book in (?, ?, ?, ?, ?, ?) \
                AND Number_of_Pages < ? \
                AND Book_Title NOT LIKE ? \
                ORDER BY RANDOM() \
                LIMIT 1", recommendations)
                results = cur.fetchall()
                
        if not results:
            return "<p> No matches. Please try again with different criteria</p>"
                
            
        email = request.get_cookie("user", secret=secretKey)    
        cur.execute("""INSERT INTO Books_Read (ID_Book, ID_User)
                    VALUES (
                    (SELECT ID_Book
                    FROM Books 
                    WHERE
                    Book_Title LIKE ?),
                    (SELECT ID_User
                    FROM Users 
                    WHERE
                    email = ?))""", ('%'+Title_of_the_book+'%', email)) 
        con.commit()
        
        con.close()
        
        recommended_title = results[0][0]
        recommended_author = results[0][1]
        recommended_year = results[0][3]
        recommended_pages = results[0][2]
        recommended_link = results[0][4]
        recommended_description = results[0][5]
        recommended_genre = results[0][6]
        recommended_rating = results[0][7]
        recommended_cover = results[0][8]
        
        
        return template('output.tpl', recommended_title= recommended_title, recommended_author = recommended_author, recommended_year = recommended_year, recommended_pages = recommended_pages, recommended_link = recommended_link, recommended_description = recommended_description, recommended_genre = recommended_genre, recommended_rating = recommended_rating, recommended_cover = recommended_cover)
    
    else:
        return template('recommendationform')
    
    
    
app.run(host='localhost', port=8090, reloader=False, debug=False)




C:\Users\marci\Desktop\bookomend


Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8090/
Hit Ctrl-C to quit.

127.0.0.1 - - [21/Jan/2021 00:31:48] "GET / HTTP/1.1" 200 1513
127.0.0.1 - - [21/Jan/2021 00:31:49] "GET /logout HTTP/1.1" 303 0
127.0.0.1 - - [21/Jan/2021 00:31:49] "GET / HTTP/1.1" 303 0
127.0.0.1 - - [21/Jan/2021 00:31:49] "GET /login HTTP/1.1" 200 1869
127.0.0.1 - - [21/Jan/2021 00:31:49] "GET /static/css/mystyle.css HTTP/1.1" 404 738
